In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🌍 Air Quality Analysis Report\n",
    "### Pipeline Summary: Trends, Forecasts & Clustering (EPA O₃)\n",
    "\n",
    "This notebook summarizes the outputs from your end-to-end ETL and analysis pipeline.\n",
    "\n",
    "#### Data Sources\n",
    "- Input file: `Cleaned_EPA_O3_Monthly.csv`\n",
    "- Feature dataset: `data_lake/feature_sets/features.parquet`\n",
    "- Trend outputs: `analysis_outputs/trends/`\n",
    "- Forecast outputs: `analysis_outputs/forecasts/`\n",
    "- Clustering outputs (if available): `analysis_outputs/clusters/`\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1️⃣ Load Libraries & Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import json, os\n",
    "from pathlib import Path\n",
    "\n",
    "sns.set(style='whitegrid', context='talk')\n",
    "\n",
    "# Define paths\n",
    "features_path = Path('data_lake/feature_sets/features.parquet')\n",
    "trend_summary_path = Path('analysis_outputs/trends/trend_summary.csv')\n",
    "forecast_summary_path = Path('analysis_outputs/forecasts/forecast_summary.csv')\n",
    "forecast_csv = Path('analysis_outputs/forecasts/forecast_O3.csv')\n",
    "forecast_plot = Path('analysis_outputs/forecasts/forecast_O3.png')\n",
    "cluster_status_path = Path('analysis_outputs/clusters/status.json') if Path('analysis_outputs/clusters/').exists() else None\n",
    "\n",
    "# Load data safely\n",
    "df = pd.read_parquet(features_path)\n",
    "trend_summary = pd.read_csv(trend_summary_path) if trend_summary_path.exists() else None\n",
    "forecast_summary = pd.read_csv(forecast_summary_path) if forecast_summary_path.exists() else None\n",
    "forecast_df = pd.read_csv(forecast_csv, parse_dates=['date']) if forecast_csv.exists() else None\n",
    "\n",
    "print('✅ Data loaded successfully!')\n",
    "print('Feature dataset shape:', df.shape)\n",
    "print('Columns:', df.columns.tolist())\n",
    "print('Unique locations:', df['location_inferred'].unique())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2️⃣ Explore Time Series Trends"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot O3 concentration over time\n",
    "plt.figure(figsize=(12,6))\n",
    "sns.lineplot(data=df, x='date', y='O3_ug_m3', label='O₃ Concentration (µg/m³)')\n",
    "plt.title('Monthly O₃ Concentration Trend')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('O₃ (µg/m³)')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "if trend_summary is not None:\n",
    "    display(trend_summary.head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3️⃣ Forecast Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if forecast_df is not None:\n",
    "    plt.figure(figsize=(12,6))\n",
    "    sns.lineplot(data=forecast_df, x='date', y='yhat', label='Forecast')\n",
    "    if 'yhat_lower' in forecast_df.columns and 'yhat_upper' in forecast_df.columns:\n",
    "        plt.fill_between(forecast_df['date'], forecast_df['yhat_lower'], forecast_df['yhat_upper'], alpha=0.3, label='Confidence Interval')\n",
    "    plt.title('10-Year Forecast (Prophet Model)')\n",
    "    plt.xlabel('Date')\n",
    "    plt.ylabel('Predicted O₃ (µg/m³)')\n",
    "    plt.legend()\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print('⚠️ Forecast data not found — please run forecasting script first.')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4️⃣ Forecast Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if forecast_summary is not None:\n",
    "    display(forecast_summary)\n",
    "else:\n",
    "    print('⚠️ forecast_summary.csv not found.')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5️⃣ Clustering Summary (if available)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if cluster_status_path and cluster_status_path.exists():\n",
    "    with open(cluster_status_path, 'r') as f:\n",
    "        cluster_status = json.load(f)\n",
    "    print('🟡 Clustering Status:', cluster_status.get('status'))\n",
    "    print('Distinct Locations:', cluster_status.get('n_locations'))\n",
    "else:\n",
    "    print('⚠️ No clustering output found (only 1 location detected).')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6️⃣ Summary of Findings"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print('📋 Summary:')\n",
    "print('- O₃ trend plotted successfully.')\n",
    "print('- Forecast generated with Prophet for 10 years.')\n",
    "print('- Clustering skipped (need ≥2 distinct locations).')\n",
    "print('- Ready to extend for multi-site datasets.')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}




NameError: name 'null' is not defined